# SAC agents

> Soft Actor Critic based agent

In [ ]:
#| default_exp agents.rl.sac

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging

# set logging level to INFO
logging.basicConfig(level=logging.INFO)

from abc import ABC, abstractmethod
from typing import Union, Optional, List, Tuple
import numpy as np
import os

from ddopnew.envs.base import BaseEnvironment
from ddopnew.agents.rl.mushroom_rl import MushroomBaseAgent
from ddopnew.utils import MDPInfo, Parameter, merge_dictionaries
from ddopnew.obsprocessors import FlattenTimeDimNumpy
from ddopnew.RL_approximators import MLPStateAction, MLPActor
from ddopnew.postprocessors import ClipAction

from ddopnew.dataloaders.base import BaseDataLoader

from mushroom_rl.algorithms.actor_critic.deep_actor_critic import SAC

import torch
import torch.nn.functional as F
from torchsummary import summary

from copy import deepcopy

import time

In [ ]:
#| export

class SACBaseAgent(MushroomBaseAgent):

    """
    XXX
    """

    def __init__(self, 
                environment_info: MDPInfo,

                # Not input regarding network architecture --> to be set in subclass

                learning_rate_actor: float = 3e-4,
                learning_rate_critic: float | None = None, # If none, then it is set to learning_rate_actor
                initial_replay_size: int = 64,
                max_replay_size: int = 50000,
                batch_size: int = 64,
                warmup_transitions: int = 100,
                lr_alpha: float = 3e-4,
                tau: float = 0.005,
                log_std_min: float = -20.0,
                log_std_max: float = 2.0,
                use_log_alpha_loss=False,
                target_entropy: float | None = None,
                
                drop_prob: float = 0.0,
                batch_norm: bool = False,
                init_method: str = "xavier_uniform", # "xavier_uniform", "xavier_normal", "he_normal", "he_uniform", "normal", "uniform"

                optimizer: str = "Adam", # "Adam" or "SGD" or "RMSprop"  
                loss: str = "MSE", # currently only MSE is supported     
                obsprocessors: list | None = None,      # default: []
                device: str = "cpu", # "cuda" or "cpu"
                agent_name: str | None = "SAC",

                network_actor_mu_params: dict = None,
                network_actor_sigma_params: dict = None,
                network_critic_params: dict = None,

                ):

        #### To set in the subclass:
        # potential pre-processor
        # input shapes for actor and critic, output shape for actor
        # actor and critic network classes
        # actor and critic network parameters

        use_cuda = self.set_device(device)

        self.warmup_training_steps = initial_replay_size

        OptimizerClass=self.get_optimizer_class(optimizer)
        learning_rate_critic = learning_rate_critic or learning_rate_actor
        lossfunction = self.get_loss_function(loss)

        actor_mu_params = dict(
                                    drop_prob=drop_prob,
                                    batch_norm=batch_norm,
                                    init_method=init_method,

                                    use_cuda=use_cuda,
                                    dropout=self.dropout
        )

        actor_sigma_params = dict(
                                    drop_prob=drop_prob,
                                    batch_norm=batch_norm,
                                    init_method=init_method,

                                    use_cuda=use_cuda,
                                    dropout=self.dropout
        )

        actor_mu_params = merge_dictionaries(actor_mu_params, network_actor_mu_params)
        actor_sigma_params = merge_dictionaries(actor_sigma_params, network_actor_sigma_params)

        actor_optimizer = {'class': OptimizerClass,
            'params': {'lr': learning_rate_actor}} 

        critic_optimizer = {'class': OptimizerClass,
            'params': {'lr': learning_rate_critic}}

        critic_params = dict(
                                optimizer = critic_optimizer,
                                loss=lossfunction,
                                drop_prob=drop_prob,
                                batch_norm=batch_norm,
                                init_method=init_method,

                                use_cuda=use_cuda,
                                dropout=self.dropout,)
                            
        critic_params = merge_dictionaries(critic_params, network_critic_params)

        self.agent = SAC(
            mdp_info=environment_info,
            actor_mu_params=actor_mu_params,
            actor_sigma_params=actor_sigma_params,
            actor_optimizer=actor_optimizer,
            critic_params=critic_params,
            batch_size=batch_size,
            initial_replay_size=initial_replay_size,
            max_replay_size=max_replay_size,
            warmup_transitions=warmup_transitions,
            tau=tau,
            lr_alpha=lr_alpha,
            use_log_alpha_loss=use_log_alpha_loss,
            log_std_min=log_std_min,
            log_std_max=log_std_max,
            target_entropy=target_entropy,
            critic_fit_params=None
        )

        super().__init__(
            environment_info=environment_info,
            obsprocessors=obsprocessors,
            device=device,
            agent_name=agent_name
        )

        logging.info("Actor network (mu network):")
        if logging.getLogger().isEnabledFor(logging.INFO):
            summary(self.actor, input_size=actor_mu_params["input_shape"])
            time.sleep(.2)

        logging.info("Critic network:")
        if logging.getLogger().isEnabledFor(logging.INFO):
            summary(self.critic, input_size=critic_params["input_shape"])

    def get_network_list(self, set_actor_critic_attributes: bool = True):
        """ Get the list of networks in the agent for the save and load functions
        Get the actor for the predict function in eval mode """

        networks = []
        ensemble_critic = self.agent._critic_approximator._impl.model
        for i, model in enumerate(ensemble_critic):
            networks.append(model.network)
        networks.append(self.agent.policy._mu_approximator._impl.model.network)
        networks.append(self.agent.policy._sigma_approximator._impl.model.network)

        actor = self.agent.policy._mu_approximator._impl.model.network
        critic = ensemble_critic[0].network

        if set_actor_critic_attributes:
            return networks, actor, critic
        else:
            return networks

    def predict_(self, observation: np.ndarray) -> np.ndarray: #
        """ Do one forward pass of the model directly and return the prediction.
        Apply tanh as implemented for the SAC actor in mushroom_rl"""

        # make observation torch tensor

        observation = torch.tensor(observation, dtype=torch.float32).to(self.device)
        action = self.actor.forward(observation)
        # print("a before tanh: ", action)
        action = torch.tanh(action)
        # print("a after tanh: ", action)
        action = action * self.agent.policy._delta_a + self.agent.policy._central_a
        # print("a after scaling: ", action)
        action = action.cpu().detach().numpy()

        return action

In [ ]:
#| export

class SACAgent(SACBaseAgent):

    """
    XXX
    """

    dropout = True # always keep in True for mushroom_RL, dropout is not desired set drop_prob=0.0

    def __init__(self, 
                environment_info: MDPInfo,

                hidden_layers: List = None, # if None, then default is [64, 64]
                activation: str = "relu", # "relu", "sigmoid", "tanh", "leakyrelu", "elu"

                learning_rate_actor: float = 3e-4,
                learning_rate_critic: float | None = None, # If none, then it is set to learning_rate_actor
                initial_replay_size: int = 64,
                max_replay_size: int = 50000,
                batch_size: int = 64,
                warmup_transitions: int = 100,
                lr_alpha: float = 3e-4,
                tau: float = 0.005,
                log_std_min: float = -20.0,
                log_std_max: float = 2.0,
                use_log_alpha_loss=False,
                target_entropy: float | None = None,

                drop_prob: float = 0.0,
                batch_norm: bool = False,
                init_method: str = "xavier_uniform", # "xavier_uniform", "xavier_normal", "he_normal", "he_uniform", "normal", "uniform"

                optimizer: str = "Adam", # "Adam" or "SGD" or "RMSprop"  
                loss: str = "MSE", # currently only MSE is supported     
                obsprocessors: list | None = None,      # default: []
                device: str = "cpu", # "cuda" or "cpu"
                agent_name: str | None = "SAC",
                ):

        # The standard SAC agent needs a 2D input, so we need to flatten the time dimension
        flatten_time_dim_processor = FlattenTimeDimNumpy(allow_2d=True, batch_dim_included=False)
        obsprocessors = (obsprocessors or []) + [flatten_time_dim_processor]

        # determine shapes
        actor_input_shape = self.get_input_shape(environment_info.observation_space) # Note: This can be a list or tuple
        actor_output_shape = environment_info.action_space.shape # Note: This can be a list or tuple
        critic_input_shape = [actor_input_shape, actor_output_shape,] # Note: This can be a list or tuple

        # Set networks (use classes, not instances)
        actor_mu_network = MLPActor
        actor_sigma_network = MLPActor
        critic_network = MLPStateAction

        # Set default for network architecture
        hidden_layers = hidden_layers or [64, 64]

        # Set network parameters
        network_actor_mu_params = dict(
                                    network = MLPActor,
                                    input_shape=actor_input_shape,
                                    output_shape=actor_output_shape,
                                    hidden_layers=hidden_layers,
                                    activation=activation,
        )

        network_actor_sigma_params = dict(
                                    network = MLPActor,
                                    input_shape=actor_input_shape,
                                    output_shape=actor_output_shape,
                                    hidden_layers=hidden_layers,
                                    activation=activation,
        )

        network_critic_params = dict(
                                    network = MLPStateAction,
                                    input_shape=critic_input_shape,
                                    output_shape=(1,),
                                    hidden_layers=hidden_layers,
                                    activation=activation,
        )

        super().__init__(
            environment_info=environment_info,

            learning_rate_actor=learning_rate_actor,
            learning_rate_critic=learning_rate_critic,
            initial_replay_size=initial_replay_size,
            max_replay_size=max_replay_size,
            batch_size=batch_size,
            warmup_transitions=warmup_transitions,
            lr_alpha=lr_alpha,
            tau=tau,
            log_std_min=log_std_min,
            log_std_max=log_std_max,
            use_log_alpha_loss=use_log_alpha_loss,
            target_entropy=target_entropy,

            drop_prob=drop_prob,
            batch_norm=batch_norm,
            init_method=init_method,

            optimizer=optimizer,
            loss=loss,
            obsprocessors=obsprocessors,
            device=device,
            agent_name=agent_name,

            network_actor_mu_params=network_actor_mu_params,
            network_actor_sigma_params=network_actor_sigma_params,
            network_critic_params=network_critic_params,
        )
        
       

In [ ]:
from ddopnew.envs.inventory import NewsvendorEnv
from ddopnew.dataloaders.tabular import XYDataLoader
from ddopnew.experiment_functions import run_experiment, test_agent

val_index_start = 8000 #90_000
test_index_start = 9000 #100_000

X = np.random.standard_normal((10000, 2))
Y = np.random.standard_normal((10000, 1))
Y += 2*X[:,0].reshape(-1, 1) + 3*X[:,1].reshape(-1, 1)
Y = X[:,0].reshape(-1, 1)
# truncate Y at 0:
Y = np.maximum(Y, 0)
# normalize Y max to 1
Y = Y/np.max(Y)

print(np.max(Y))

print(X.shape, Y.shape)

clip_action = ClipAction(0., 1.)

dataloader = XYDataLoader(X, Y, val_index_start, test_index_start, lag_window_params =  {'lag_window': 0, 'include_y': False, 'pre_calc': True})

environment = NewsvendorEnv(
    dataloader = dataloader,
    underage_cost = 0.42857,
    overage_cost = 1.0,
    gamma = 0.999,
    horizon_train = 365,
    q_bound_high = 1.0,
    q_bound_low = -0.1,
    postprocessors = [clip_action],
)


agent = SACAgent(environment.mdp_info,
                obsprocessors = None,      # default: []
                device="cpu", # "cuda" or "cpu"
)

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

environment.train()
agent.train()
environment.print=False

# run_experiment(agent, environment, n_epochs=50, n_steps=1000, run_id = "test", save_best=True, print_freq=1) # fit agent via run_experiment function

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

1.0
(10000, 2) (10000, 1)


/Users/magnus/miniforge3/envs/inventory_gym_2/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


IndexError: invalid index to scalar variable.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()